In [ ]:
# STEP 1: Set up the environment
%cd /content/
!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

# STEP 2: Install required Python packages
!pip install -r requirements.txt

# STEP 3: Build llama.cpp using CMake (IMPORTANT)
!mkdir -p build
%cd build
!cmake ..
!cmake --build . --config Release

# STEP 4: Confirm binaries were built
!ls /content/llama.cpp/build/bin/

/content
Cloning into 'llama.cpp'...
remote: Enumerating objects: 48989, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 48989 (delta 98), reused 55 (delta 50), pack-reused 48826 (from 3)
Receiving objects: 100% (48989/48989), 103.01 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (35271/35271), done.
/content/llama.cpp
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This 

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_repo = "ShivomH/Elixir-Health-Llama3B"
local_dir = "/content/Elixir-Health-Llama3b"
snapshot_download(repo_id=model_repo, local_dir=local_dir, ignore_patterns=["*.bin.index.json"])

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

'/content/Elixir-Health-Llama3b'

In [ ]:
# Convert to GGUF
%cd /content/llama.cpp
!python3 convert_hf_to_gguf.py /content/Elixir-Health-Llama3b --outfile /content/elixir-health.gguf

/content/llama.cpp
INFO:hf-to-gguf:Loading model: Elixir-Health-Llama3b
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {3072, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfl

In [ ]:
# Quantize to 4-bit (Q4_K_M) and 8-bit (Q8_0)
!./build/bin/llama-quantize /content/elixir-health.gguf /content/elixir-health-q4.gguf Q4_K_M
!./build/bin/llama-quantize /content/elixir-health.gguf /content/elixir-health-q8.gguf Q8_0

main: build = 5171 (2cca6c01)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/elixir-health.gguf' to '/content/elixir-health-q4.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 30 key-value pairs and 256 tensors from /content/elixir-health.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                    

In [ ]:
#Upload to Hugging Face
from huggingface_hub import HfApi, upload_file

In [ ]:
api = HfApi()
repo_id=("ShivomH/Elixir-Health-Quantized")
api.create_repo(repo_id=repo_id, repo_type="model", private=False)

RepoUrl('https://huggingface.co/ShivomH/Elixir-Health-Quantized', endpoint='https://huggingface.co', repo_type='model', repo_id='ShivomH/Elixir-Health-Quantized')

In [ ]:
# Upload 4-bit
upload_file(
    path_or_fileobj="/content/elixir-health-q4.gguf",
    path_in_repo="elixir-health-q4.gguf",
    repo_id=repo_id,
    repo_type="model"
)

elixir-health-q4.gguf:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShivomH/Elixir-Health-Quantized/commit/6a04bbca4d7173b3e59db52a5b8857b546458b2b', commit_message='Upload elixir-health-q4.gguf with huggingface_hub', commit_description='', oid='6a04bbca4d7173b3e59db52a5b8857b546458b2b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Upload 8-bit
upload_file(
    path_or_fileobj="/content/elixir-health-q8.gguf",
    path_in_repo="elixir-health-q8.gguf",
    repo_id=repo_id,
    repo_type="model"
)

elixir-health-q8.gguf:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShivomH/Elixir-Health-Quantized/commit/511d6dd7749713d6908c65813130fcaa5affa660', commit_message='Upload elixir-health-q8.gguf with huggingface_hub', commit_description='', oid='511d6dd7749713d6908c65813130fcaa5affa660', pr_url=None, pr_revision=None, pr_num=None)